### **Generating all possible solutions for [mathler](https://mathler) game** *([Wordle](https://www.nytimes.com/games/wordle/index.html)'s mathematic variant)*

First of all, I thought this wouldn't be feasable, that the ammount of possible equations and results would be to large to compute. But I wanted to try it anyways, and it turns out, there aren't that many valid possible equations.

Let's list the explicit rules for each possible solution:

+ Each equation must be exactly 6 chars long *(each char is considered to be a single digit number or an operator)*
+ Numbers and operators can appear more than once
+ No leading zeros are allowed
+ Calculations follow the order of operations **PEDMAS**
    1. Parentheses
    2. Exponents
    3. Division or Multiplication
    4. Addition or Subtraction

We can also extrapolate some other rules based on the previous requirements:

+ Equations can start with a `'-'` sign, but can't start with any other operator *(`'/','*','+'`)*
+ Equations can't end with any of the operators *(`'/','*','+','-'`)*
+ Only divisions that result in an integer are valid
+ Divisions by zero are not allowed
+ There can't be equations that are equal to the solution:
  + 6 digit numbers
  + 5 digit negative numbers

In [24]:
from tqdm import tqdm
import itertools
import csv
import re
import os

from solver import solve


# Definining constants
EQUATIONS_FILEPATH = "data\\equations.csv"
ELEMENTS_PATTERN = [
    "1234567890-",
    "1234567890/*+-",
    "1234567890/*+-",
    "1234567890/*+-",
    "1234567890/*+-",
    "1234567890",
]


# Defining helper functions
def print_preview(equations):
    """Print the first and last elements of a list and it's length"""

    print(f"First elements:\t{equations[:5]}")
    print(f"Last elements:\t{equations[-5:]}")
    print(f"Length:\t\t{len(equations):,}")


def write_csv(filepath, equations):
    """Writes rows to a csv file"""

    if os.path.exists(filepath):
        os.remove(filepath)

    with open(filepath, "a", newline="") as file:
        writer = csv.writer(file)

        for equation in tqdm(equations, desc="Writing equations to csv"):
            writer.writerow(equation)

We will now generate all combinations of elements and convert them to `strings`

In [25]:
# List of possible elements for each position of the equation
equations = list(itertools.product(*ELEMENTS_PATTERN))

# Convert equations to strings
equations = list(map(lambda x: "".join(x), tqdm(equations)))

# Print a preview
print_preview(equations)

100%|██████████| 4225760/4225760 [00:01<00:00, 2654582.15it/s]


First elements:	['111111', '111112', '111113', '111114', '111115']
Last elements:	['-----6', '-----7', '-----8', '-----9', '-----0']
Length:		4,225,760


Discard equations containing consecutive operators.

In [26]:
# Defining the Regex pattern
pattern = r"[/,*,+,-][/,*,+,-]"

# Looking for the first occurance of the pattern
for equation in equations:
    if re.findall(pattern, equation):
        print(equation)
        break

# Discard equations that contain the pattern
equations = [x for x in tqdm(equations) if not re.findall(pattern, x)]

# Print a preview
print_preview(equations)

111//1


100%|██████████| 4225760/4225760 [00:04<00:00, 1017667.12it/s]


First elements:	['111111', '111112', '111113', '111114', '111115']
Last elements:	['-0-0-6', '-0-0-7', '-0-0-8', '-0-0-9', '-0-0-0']
Length:		3,316,000


Discard equations containing a single number *(6 digit positive numbers and 5 digit negative numbers)*.

In [27]:
# Defining the Regex pattern
pattern = r"[0-9,-][0-9]{5}"

# Looking for the first occurance of the pattern
for equation in equations:
    if re.findall(pattern, equation):
        print(equation)
        break

# Discard equations that contain the pattern
equations = [x for x in tqdm(equations) if not re.findall(pattern, x)]

# Print a preview
print_preview(equations)

111111


100%|██████████| 3316000/3316000 [00:03<00:00, 887354.85it/s] 


First elements:	['1111/1', '1111/2', '1111/3', '1111/4', '1111/5']
Last elements:	['-0-0-6', '-0-0-7', '-0-0-8', '-0-0-9', '-0-0-0']
Length:		2,216,000


Discard equations containing leading zeros.

In [28]:
# Defining the Regex pattern
pattern = r"0\d"

# Looking for the first occurance of the pattern
for equation in equations:
    if re.findall(pattern, equation):
        print(equation)
        break

# Discard equations that contain the pattern
equations = [x for x in tqdm(equations) if not re.findall(pattern, x)]

# Print a preview
print_preview(equations)

111/01


100%|██████████| 2216000/2216000 [00:02<00:00, 973635.10it/s] 

First elements:	['1111/1', '1111/2', '1111/3', '1111/4', '1111/5']
Last elements:	['-0-0-6', '-0-0-7', '-0-0-8', '-0-0-9', '-0-0-0']
Length:		1,711,600


Discard equations containing divisions by zero.

In [29]:
# Defining the Regex pattern
pattern = r"/0"

# Looking for the first occurance of the pattern
for equation in equations:
    if re.findall(pattern, equation):
        print(equation)
        break

# Discard equations that contain the pattern
equations = [x for x in tqdm(equations) if not re.findall(pattern, x)]

# Print a preview
print_preview(equations)

1111/0


100%|██████████| 1711600/1711600 [00:01<00:00, 1036810.42it/s]

First elements:	['1111/1', '1111/2', '1111/3', '1111/4', '1111/5']
Last elements:	['-0-0-6', '-0-0-7', '-0-0-8', '-0-0-9', '-0-0-0']
Length:		1,688,400


Discard equations that have invalid solutions. Then, sort by solution.

In [30]:
temp = []

for equation in tqdm(equations):
    solution = solve(equation)

    if solution:
        temp.append((equation, solution))

# Sort by solution
equations = sorted(temp, key=lambda x: x[1])

# Print a preview
print_preview(equations)

100%|██████████| 1688400/1688400 [00:23<00:00, 73389.06it/s]


First elements:	[('0-9999', -9999), ('1-9999', -9998), ('0-9998', -9998), ('1-9998', -9997), ('2-9999', -9997)]
Last elements:	[('99*997', 98703), ('998*99', 98802), ('99*998', 98802), ('999*99', 98901), ('99*999', 98901)]
Length:		1,240,278


Check the generated equations against their solutions, making use of Python's built in function `eval()`.

In [31]:
for equation, solution in tqdm(equations):
    if eval(equation) != solution:
        print(f"{equation} = {solution}")
        break

100%|██████████| 1240278/1240278 [00:10<00:00, 122293.68it/s]


Write equations and their solutions to a `.csv` file.

In [32]:
write_csv(EQUATIONS_FILEPATH, equations)

Writing equations to csv: 100%|██████████| 1240278/1240278 [00:02<00:00, 448970.61it/s]
